In [3]:
!pip install pytorch_transformers==1.0

load model and use it(need to download the model to save in own computer in huggingface
https://huggingface.co/gpt2/tree/main

In [4]:
import torch
from pytorch_transformers import GPT2Tokenizer

import logging
logging.basicConfig(level=logging.INFO)

# Load a pre-trained model of the word splitter
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

# Encode the input using GPT2Tokenizer
text = "Yesterday, a man named Jack said he saw an alien,"
indexed_tokens = tokenizer.encode(text)
tokens_tensor = torch.tensor([indexed_tokens])
tokens_tensor.shape

INFO:pytorch_transformers.tokenization_utils:loading file https://s3.amazonaws.com/models.huggingface.co/bert/gpt2-vocab.json from cache at C:\Users\ginny\.cache\torch\pytorch_transformers\f2808208f9bec2320371a9f5f891c184ae0b674ef866b79c58177067d15732dd.1512018be4ba4e8726e41b9145129dc30651ea4fec86aa61f4b9f40bf94eac71
INFO:pytorch_transformers.tokenization_utils:loading file https://s3.amazonaws.com/models.huggingface.co/bert/gpt2-merges.txt from cache at C:\Users\ginny\.cache\torch\pytorch_transformers\d629f792e430b3c76a1291bb2766b0a047e36fae0588f9dbc1ae51decdff691b.70bec105b4158ed9a1747fea67a43f5dee97855c64d62b6ec3742f4cfdb5feda


torch.Size([1, 12])

In [5]:
with open('data/clean_shortjokes.txt', 'r') as f:
    dataset = f.read()

len(dataset)

5915159

In [6]:
indexed_text = tokenizer.encode(dataset)
del(dataset)

dataset_cut = []
for i in range(len(indexed_text)//60):
     # Segment the string to a length of 60
    dataset_cut.append(indexed_text[i*60:i*60+60])
del(indexed_text)

dataset_tensor = torch.tensor(dataset_cut)
dataset_tensor.shape

torch.Size([25610, 60])

In [7]:
from torch.utils.data import DataLoader, TensorDataset

# Build dataset and data iterator, set batch_size size to 2
train_set = TensorDataset(dataset_tensor,
                          dataset_tensor)  # Labels are the same as the sample data
train_loader = DataLoader(dataset=train_set,
                          batch_size=2,
                          shuffle=False)
train_loader

Use CPU to training

In [8]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cpu')

In [9]:
from pytorch_transformers import GPT2LMHeadModel
model= GPT2LMHeadModel.from_pretrained("C:/Users/ginny/pythonStudy/gpt-2-master/model")# Here is the address of the GPT-2 model file, I saved it on my own computer and did not pack it into the assignment folder because it was too large

INFO:pytorch_transformers.modeling_utils:loading configuration file C:/Users/ginny/pythonStudy/gpt-2-master/model\config.json
INFO:pytorch_transformers.modeling_utils:Model config {
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "finetuning_task": null,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 768,
  "n_head": 12,
  "n_layer": 12,
  "n_positions": 1024,
  "num_labels": 1,
  "output_attentions": false,
  "output_hidden_states": false,
  "resid_pdrop": 0.1,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "summary_use_proj": true,
  "task_specific_params": {
    "text-generation": {
      "do_sample": true,
      "max_length": 50
    }
  },
  "torchscript": false,
  "vocab_size": 50257
}

INFO:pytorch_transfo

training:

In [ ]:
from torch import nn
from torch.autograd import Variable
import time

pre = time.time()

epoch = 2  # Cycle study 2 times

model.to(device)
model.train()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-5)  # Define Optimizer

for i in range(epoch):
    total_loss = 0
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = Variable(data).to(device), Variable(
            target).to(device)

        optimizer.zero_grad()

        loss, logits, _ = model(data, labels=target)

        total_loss += loss

        loss.backward()
        optimizer.step()

        if batch_idx == len(train_loader)-1:
            # Output the results at the end of each Epoch
            print('average loss:', total_loss/len(train_loader))

print('training time：', time.time()-pre)

generate texts

In [ ]:
text = "A dog"  # Here you can also enter different English texts
indexed_tokens = tokenizer.encode(text)
tokens_tensor = torch.tensor([indexed_tokens])

model.eval()
total_predicted_text = text

# Make 20 predictions from the trained model
for _ in range(20):
    tokens_tensor = tokens_tensor.to('cuda')

    with torch.no_grad():
        outputs = model(tokens_tensor)
        predictions = outputs[0]

    predicted_index = select_top_k(predictions, k=10)

    predicted_text = tokenizer.decode(indexed_tokens + [predicted_index])
    total_predicted_text += tokenizer.decode(predicted_index)
    if '<|endoftext|>' in total_predicted_text:
        # If the end-of-text flag appears, end text generation
        break

    indexed_tokens += [predicted_index]

    if len(indexed_tokens) > 60:
        # Maximum model input length is 60, truncated if length is too long
        indexed_tokens = indexed_tokens[-60:]

    tokens_tensor = torch.tensor([indexed_tokens])

print(total_predicted_text)